# Seq2Seq Models

The seq2seq model is a specialized deep learning architecture that makes use of LSTM layers to translate on sequence/pattern into another sequence pattern.  Practical applications include machine translation, text generation, music generation, or potentially text-to-speech or speech-to-text.

In [1]:
import re, random

import numpy as np
import pandas as pd

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM
from tensorflow.keras.optimizers import Adam

References:

- https://www.kaggle.com/shujunge/lstm-seq2seq-with-keras/notebook
- https://towardsdatascience.com/generative-chatbots-using-the-seq2seq-model-d411c8738ab5
- https://machinelearningmastery.com/define-encoder-decoder-sequence-sequence-model-neural-machine-translation-keras/

## Data Pre-processing

In [2]:
path = '../data/fra-eng/fra.txt'

In [3]:
input_texts = []
target_texts = []
input_chars = set()
target_chars = set()
with open(path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [4]:
for line in lines[:10000]:
    input_text = line.split('\t')[0]
    target_text = line.split('\t')[1]
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_chars:
            input_chars.add(char)
    for char in target_text:
        if char not in target_chars:
            target_chars.add(char)

input_chars = sorted(list(input_chars))
target_chars = sorted(list(target_chars))
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)
max_encoder_seq_len = max([len(txt) for txt in input_texts])
max_decoder_seq_len = max([len(txt) for txt in target_texts])

In [5]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_len)
print('Max sequence length for outputs:', max_decoder_seq_len)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [6]:
# Token Lookups
input_token_index = dict([(char, i) for i, char in enumerate(input_chars)])
target_token_index = dict([(char, i) for i, char in enumerate(target_chars)])

# One-hot Vectors
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_len, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_len, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_len, num_decoder_tokens), dtype='float32')

# Adjust for timesteps
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [7]:
encoder_input_data[:2]

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)

<img src="https://miro.medium.com/max/822/1*uvCHgbPgvVxvW2pGrJozBA.png">

Above visualization is from: https://towardsdatascience.com/generative-chatbots-using-the-seq2seq-model-d411c8738ab5

## Modeling

In [8]:
# Hyperparameters
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000

In [9]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, hidden_state, cell_state = encoder(encoder_inputs)
encoder_states = [hidden_state, cell_state]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [10]:
model.summary()
print(model.layers[-3].output)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 71)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 92)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 335872      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  357376      input_2[0][0]                    
                                                                 lstm[0][1]                   

In [11]:
print("encoder_input_data shape:",encoder_input_data.shape)
print("decoder_input_data shape:",decoder_input_data.shape)
print("decoder_target_data shape:",decoder_target_data.shape)

encoder_input_data shape: (10000, 15, 71)
decoder_input_data shape: (10000, 59, 92)
decoder_target_data shape: (10000, 59, 92)


In [12]:
model.compile(optimizer=Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, decay=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 32s 253ms/step - loss: 0.2197 - accuracy: 0.2412 - val_loss: 0.5170 - val_accuracy: 0.1989
Epoch 2/100
125/125 [==============================] - 32s 254ms/step - loss: 0.2186 - accuracy: 0.2415 - val_loss: 0.5153 - val_accuracy: 0.1992
Epoch 3/100
125/125 [==============================] - 31s 246ms/step - loss: 0.2179 - accuracy: 0.2418 - val_loss: 0.5156 - val_accuracy: 0.1992
Epoch 4/100
125/125 [==============================] - 31s 246ms/step - loss: 0.2171 - accuracy: 0.2419 - val_loss: 0.5202 - val_accuracy: 0.1974
Epoch 5/100
125/125 [==============================] - 30s 239ms/step - loss: 0.2159 - accuracy: 0.2426 - val_loss: 0.5184 - val_accuracy: 0.1984
Epoch 6/100
125/125 [==============================] - 30s 241ms/step - loss: 0.2149 - accuracy: 0.2427 - val_loss: 0.5183 - val_accuracy: 0.1986
Epoch 7/100
125/125 [==============================] - 31s 246ms/step - loss: 0.2140 - accuracy: 0.2431 - val_loss: 0.5193 -

In [19]:
# Generative Model
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [23]:
reverse_input_char_idx = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_idx = dict((i, char) for char, i in target_token_index.items())

In [24]:
def decode(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_idx[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_len):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [25]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Allez-vous.

-
Input sentence: Go.
Decoded sentence: Allez-vous.

-
Input sentence: Go.
Decoded sentence: Allez-vous.

-
Input sentence: Hi.
Decoded sentence: Calme tardin.

-
Input sentence: Hi.
Decoded sentence: Calme tardin.

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded 